### 확률적 경사하강법 (SGD: Stocastic Gradient Descent)
- 머신러닝에서도 많이 사용하지만 실질적으로 딥러닝에서 많이 사용한다.
- 데이터가 추가되었을 경우의 머신러닝 처리 방법
1. 새로운 데이터로 모델을 추가하여 새롭게 학습하여 처리한다.
2. 추가된 개수만큼 기존의 데이터를 버리고 사용한다.
3. 기존 모델을 수정하면서 사용한다. (점진적 학습방법)

- Epoch: 훈련세트를 다 사용했을 경우의 단계
- mini batch 경사하강법 중 mini batch의 개수: Hyper Parameter, 보통 2의 배수

### 손실함수
- 머신러닝 알고리즘이 에러율을 측정하는 함수
- 손실함수 값이 적은 것이 정확도가 높은 것

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
fish = pd.read_csv('../Data/fishes.csv')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [3]:
# Feature
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
# Target
fish_target = fish['Species'].to_numpy()

In [4]:
# Train, Test
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input,fish_target,random_state=42)

In [5]:
# 표준화
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [6]:
# 확률적 경사하강법의 분류 알고리즘 적용
from sklearn.linear_model import SGDClassifier

In [7]:
sc = SGDClassifier(loss='log',max_iter=10, random_state=42)

In [8]:
sc.fit(train_scaled,train_target)
print("Train:",sc.score(train_scaled,train_target))
print("Test:",sc.score(test_scaled,test_target))

Train: 0.773109243697479
Test: 0.775


> score가 별로 안 좋고, 과소적합

In [9]:
sc.partial_fit(train_scaled,train_target)
print("Train:",sc.score(train_scaled,train_target))
print("Test:",sc.score(test_scaled,test_target))

Train: 0.8151260504201681
Test: 0.85


---
### Epoch의 과대/과소적합
- 규제 값이 작아지면 과대적합이 된다.
- 조기종료 (Early Stopping): 적합한 Epoch의 위치를 찾으면 종료한다.

In [10]:
sc = SGDClassifier(loss='log',random_state=42)
train_score = []
test_score = []

classes = np.unique(train_target)
classes

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

In [11]:
for _ in range(0,300):
    sc.partial_fit(train_scaled,train_target,classes=classes)
    train_score.append(sc.score(train_scaled,train_target))
    test_score.append(sc.score(test_scaled,test_target))

> 시각화

> 100정도의 Epoch가 좋은 것 같다

In [12]:
# 새롭게
sc = SGDClassifier(loss='log',max_iter=100,random_state=42, tol=None)

# tol = None: 조기종료 시점, 현재는 정지기준 없음, 반복기간동안 최적의 손실값보다 큰 경우 종료된다.
sc.fit(train_scaled,train_target)
print('Train:',sc.score(train_scaled,train_target))
print('Test:',sc.score(test_scaled,test_target))

Train: 0.957983193277311
Test: 0.925


In [13]:
# 조기종료 시점 적용
sc = SGDClassifier(loss='log',max_iter=100,random_state=42, tol=1.0e-3)

sc.fit(train_scaled,train_target)
print('Train:',sc.score(train_scaled,train_target))
print('Test:',sc.score(test_scaled,test_target))

Train: 0.8403361344537815
Test: 0.8


---
### SVM을 이용하는 확률적 경사 하강법

In [14]:
# 조기종료가 없는 경우
sc = SGDClassifier(loss='hinge',max_iter=100,random_state=42, tol=None)

sc.fit(train_scaled,train_target)
print('Train:',sc.score(train_scaled,train_target))
print('Test:',sc.score(test_scaled,test_target))

Train: 0.9495798319327731
Test: 0.925


In [15]:
# 조기종료가 있는 경우
sc = SGDClassifier(loss='hinge',max_iter=100,random_state=42, tol=1.0e-3)

sc.fit(train_scaled,train_target)
print('Train:',sc.score(train_scaled,train_target))
print('Test:',sc.score(test_scaled,test_target))

Train: 0.907563025210084
Test: 0.925
